<a href="https://colab.research.google.com/github/JgibsonAB/Wine_capstone/blob/main/Wine_initial_appmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Some of this processing will be similar to the first tf-idf model. This will be far more streamlined for the purposes of making a workable model for the app to use.**

In [10]:
!pip install -U sentence-transformers
import numpy as np 
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

In [23]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
#helper function
#removes punctuations and stop words
def process_sentences(sentences):
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(sentences)
    tokenized_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    remove_punctuation = [word for word in tokenized_sentence if word.isalnum()]
    cleaned_text = ''
    for word in remove_punctuation:
        cleaned_text = cleaned_text +' ' + word 
    return cleaned_text

In [13]:

wine_data = pd.read_csv('gdrive/My Drive/winemag-data-130k-v2.csv')

In [14]:
# quick look at the original dataset
wine_data.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [15]:
#data cleaning and formatting. We will create selectable features,
# most common countries (*Canada is included as it represents higher sales in North America then represented),
#sets of price points, review score sets. Otherwise set the uncommon to other.
countries = ['US','France','Italy','Spain','Portugal','Chile','Argentina','Australia','Canada', 'Germany']
for i in range(len(wine_data['country'])):
    if not (wine_data['country'][i] in countries):
        wine_data.loc[i, 'country'] = 'Other'

wine_data['points'] = pd.cut(wine_data['points'], bins=[79,85,90,95,100], labels=['79-85','85-90','90-95','95-100'])

selected_provinces = ['California', 'Bordeaux', 'Washington','Oregon','Tuscany', 'Sicily & Sardinia','Mendoza Province','Piedmont','Northeastern Italy','Northern Spain', 'South Australia' ]
for i in range(len(wine_data['province'])):
    if not (wine_data['province'][i] in selected_provinces):
        wine_data.loc[i, 'province'] = 'Other'
        
wine_data['price'] = pd.cut(wine_data['price'], bins=[0,10,20,30,60,1000], labels=['0-10','10-20','20-30','30-60', '>60'])

In [17]:
useful_features = ['country','points','price','province' ]
temp = wine_data[useful_features]
temp['any_country'] = 1
temp['any_points'] = 1
temp['any_price'] = 1
temp['any_province'] = 1
wine_f = pd.get_dummies(temp, prefix='', prefix_sep='')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [18]:
wine_f.columns

Index(['any_country', 'any_points', 'any_price', 'any_province', 'Argentina',
       'Australia', 'Canada', 'Chile', 'France', 'Germany', 'Italy', 'Other',
       'Portugal', 'Spain', 'US', '79-85', '85-90', '90-95', '95-100', '0-10',
       '10-20', '20-30', '30-60', '>60', 'Bordeaux', 'California',
       'Mendoza Province', 'Northeastern Italy', 'Northern Spain', 'Oregon',
       'Other', 'Piedmont', 'Sicily & Sardinia', 'South Australia', 'Tuscany',
       'Washington'],
      dtype='object')

In [33]:
wine_f.to_csv('gdrive/My Drive/non_description_matrix', index=False)

In [20]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
#group together descriptions for encodeing and then encode
des_list = []
for des in wine_data.description:
    des_list.append(process_sentences(des))
des_vectors = model.encode(des_list)

In [26]:
#test the model on some text inputs to see if it works. this is word review associated portion of the model.
input_text = ' A bold, but fruit forward wine with oak.'
processed_input = model.encode(process_sentences(input_text))
#dotted_vec = np.dot(des_vector, processed_input)
dotted_vec = util.cos_sim(des_vectors, processed_input)
index = np.array(dotted_vec).T[0].argsort()[-5:][::-1] #get top 3 matches

recommendation = wine_data.loc[index].description
print(recommendation)

28995     Thick and chocolaty, this wine is medium in we...
92366     This wine's alcohol is apparent from the start...
63586     This wine has a texture of softness and juicin...
108581    This is a ripe, tropical and quite flabby wine...
69799     This wine is tart with dill and cedar notes, s...
Name: description, dtype: object


In [28]:
wine_data.description[500]

"Aromas of watermelon and a dusting of natural vanilla mark the bouquet. On the palate, it's fleshy but still crisp and focused, with nectarine, red apple and strawberry flavors. The finish lasts, with some sweetness."

In [32]:
#save 
final_description_matrix = pd.DataFrame(des_vectors).to_csv('gdrive/My Drive/final_description_matrix', index=False)
